In [1]:
import sys
sys.path.append('..')

%load_ext autoreload
%autoreload 2

In [55]:
pn.extension()

In [58]:
import param

from pyllments.base.model_base import Model
from pyllments.payloads.message import MessagePayload

class ChatInterfaceModel(Model):
    message_list = param.List(default=[], instantiate=True)
    persist = param.Boolean(default=False, instantiate=True)
    new_messages = param.List(item_type=MessagePayload, doc="""
        Temporary list of new messages to be added to the message list.
        Needs to be triggered to register an event due to mutability of list""")
    new_messages_event = param.Event()
    
    def __init__(self, **params):
        super().__init__(**params)

        self.create_watchers()

    def create_watchers(self):
        self.param.watch(self._new_messages_updated, 'new_messages')

    def _new_messages_updated(self, event):
        print(event)
        # self.message_list.extend(new)
        # self.new_messages = []
        # self.message_list.append(('human', 'what is the capital of the moon?'))


In [59]:
import panel as pn
import param

from pyllments.base.element_base import Element
from pyllments.base.model_base import Model
# from pyllments.elements.chat_interface import ChatInterfaceModel
from pyllments.payloads.message import MessagePayload

class ChatInterfaceElement(Element):
    """
    Model:
    - messages in the chat
    - message input
    Views:
    - chat feed: 
    - chat input
    - send button:
    Ports:
    - input:
        - MessagePayload
    - output port
        - MessagePayload
    """
    model = param.ClassSelector(
        class_=Model,
        is_instance=True,
        default=ChatInterfaceModel()
    )

    persist = param.Boolean(default=False, instantiate=True, doc="""
        Whether to persist messages to disk between sessions.""")

    chatfeed_view = param.ClassSelector(class_=pn.chat.ChatFeed, is_instance=True)
    chat_input_view = param.ClassSelector(class_=pn.chat.ChatAreaInput, is_instance=True)
    send_button_view = param.ClassSelector(class_=pn.widgets.Button, is_instance=True)

    def __init__(self, **params):
        super().__init__(**params)
        self.model.persist = self.persist
        
        # self.ports.add_input('message_input',)
        # self.ports.add_ouput('message_output', )

    def _message_input_unpack_payload(self, payload: MessagePayload):

        return payload

    def _message_output_pack_payload(self,) -> MessagePayload:
        pass
    
    def create_chatfeed_view(self, **kwargs):
        """
        Creates and returns a new instance of ChatFeed view.
        """
        self.chatfeed_view = pn.chat.ChatFeed(**kwargs)
        return self.chatfeed_view
    
    def create_chat_input_view(self, **kwargs):
        """
        Creates and returns a new instance of ChatAreaInput view.
        """
        self.chat_input_view = pn.chat.ChatAreaInput(**kwargs)
        return self.chat_input_view

    def create_send_button_view(self, **kwargs):
        """
        Creates and returns a new instance of Button view for sending messages.
        """
        self.send_button_view = pn.widgets.Button(name='send', **kwargs)
        self.send_button_view.on_click(self._on_send)

        return self.send_button_view

    def _on_send(self, event):
        """
        Handles the send button event by appending the user's message to the chat model,
        clearing the input field, and updating the chat feed view.
        """
        print(event)
        if self.chat_input_view:
            self.model.new_messages.append(self.chat_input_view.value)
            self.model.param.trigger('new_messages')
            self.chat_input_view.value = ''
        # self.chatfeed_view.update()


In [60]:
chat_interface_element = ChatInterfaceElement()
chat_input_view = chat_interface_element.create_chat_input_view()
chat_input_view.value = 'hello'

create_send_button_view = chat_interface_element.create_send_button_view()
pn.Row(chat_input_view, create_send_button_view).servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'e7c53e4e-4041-42d0-bed1-7903323c0eb1': {'version…

Event(what='value', name='clicks', obj=Button(clicks=1, name='send'), cls=Button(clicks=1, name='send'), old=0, new=1, type='set')


TypeError: List parameter 'ChatInterfaceModel.new_messages' items must be instances of <class 'pyllments.payloads.message.message_payload.MessagePayload'>, not <class 'str'>.

In [57]:
chat_interface_element.model.param

Name,Value,Type,Range
created_message,None,ClassSelector,nullable MessagePayload
message_list,"[('human', 'what is the capi..",List,"(0, None)"
name,'ChatInterfaceModel00181',String,nullable constant
